In [ ]:
# https://github.com/calmisential/TensorFlow2.0_ResNet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp -rf /content/drive/MyDrive/colab/didimdol/* .
!unzip  /content/dataset/dataset.zip -d ./dataset

In [8]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.applications import resnet50
from tensorflow.keras import preprocessing
from tensorflow.keras.callbacks import ModelCheckpoint

In [1]:
from model.tensorflow.resnet50.model import resnet_50

In [2]:
model = resnet_50(num_classes=1)

In [3]:
model.summary()

Model: "res_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  9472      
_________________________________________________________________
batch_normalization (BatchNo multiple                  256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
sequential (Sequential)      (None, 56, 56, 64)        151296    
_________________________________________________________________
sequential_4 (Sequential)    (None, 28, 28, 128)       780288    
_________________________________________________________________
sequential_9 (Sequential)    (None, 14, 14, 128)       1195008   
_________________________________________________________________
sequential_16 (Sequential)   (None, 7, 7, 256)         2309

# 모델 로딩 및 binary classification 세팅

In [52]:
model = resnet50.ResNet50(weights=None, classes = 1, classifier_activation='sigmoid')

In [55]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_7[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [19]:
x = keras.Input(shape=(None, None, 3))
model = keras.Model(x, model)

ValueError: Output tensors of a Functional model must be the output of a TensorFlow `Layer` (thus holding past layer metadata). Found: <keras.engine.sequential.Sequential object at 0x7efe5c0e0940>

In [13]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

# 학습데이터 셋업

In [11]:
trn_data_dir = 'dataset/casting_data/train'
val_data_dir = 'dataset/casting_data/test'
batch_size = 64
img_height = 224
img_width = 224

In [12]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    trn_data_dir,
    label_mode = 'binary',
    seed=123,
    shuffle=True,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_data_dir,
    #validation_split=0.2,
    label_mode = 'binary',
    #subset="validation",
    shuffle=False,
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 6633 files belonging to 2 classes.
Found 715 files belonging to 2 classes.


In [13]:
data_augmentation = keras.Sequential(
  [
    keras.layers.experimental.preprocessing.Rescaling(1./255),
    keras.layers.experimental.preprocessing.Normalization(mean=[0.485, 0.456, 0.406], variance=[0.229, 0.224, 0.225]),
    keras.layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(img_height, img_width, 3)),
    keras.layers.experimental.preprocessing.RandomFlip("vertical",   input_shape=(img_height, img_width, 3)),
  ]
)

test_processing = keras.Sequential(
  [
    keras.layers.experimental.preprocessing.Rescaling(1./255),
    keras.layers.experimental.preprocessing.Normalization(mean=[0.485, 0.456, 0.406], variance=[0.229, 0.224, 0.225])
  ]
)

normalized_trn_ds = train_ds.map(lambda x, y: (data_augmentation(x), y))
normalized_test_ds = test_ds.map(lambda x, y: (test_processing(x), y))

# 모델 학습 셋업

In [14]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.BinaryAccuracy()])

In [15]:
checkpoint = ModelCheckpoint('check_points/resnet50/model_lite.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

# 모델 학습

In [16]:
epochs = 1
history = model.fit(
  normalized_trn_ds,
  validation_data=normalized_test_ds,
  epochs=epochs,
  # callbacks=callbacks_list
)

104/104 [==============================] - 50s 423ms/step - loss: 0.2816 - binary_accuracy: 0.8765 - val_loss: 18.6309 - val_binary_accuracy: 0.3664


In [18]:
model.save_weights('check_points/resnet50/model_lite.h5')

In [5]:
model.save('test.h5', save_format="tf")

NotImplementedError: Saving the model to HDF5 format requires the model to be a Functional model or a Sequential model. It does not work for subclassed models, because such models are defined via the body of a Python method, which isn't safely serializable. Consider saving to the Tensorflow SavedModel format (by setting save_format="tf") or using `save_weights`.